### Import requires libraries
This tutorial uses just Python core libraries, Scipy (Numpy, Matplotlib and others) and GDAL. For Windows users, the most effective way is to download the GDAL Wheel from https://www.lfd.uci.edu/~gohlke/pythonlibs/ and install through pip.

In [5]:
from osgeo import ogr, gdal, osr
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import fnmatch
from PIL import Image
import os, os.path
from pylab import *
from parse import parse
import datetime
import pandas as pd
import skimage.transform as st

In [6]:
def saveRaster(dataset,datasetPath,cols,rows,projection,geotransform):
    rasterSet = gdal.GetDriverByName('GTiff').Create(datasetPath, cols, rows,1,gdal.GDT_Float32)
    rasterSet.SetProjection(projection)
    rasterSet.SetGeoTransform(geotransform)
    rasterSet.GetRasterBand(1).WriteArray(dataset)
    rasterSet.GetRasterBand(1).SetNoDataValue(-999)
    rasterSet = None

In [7]:
def plotNDVI(ndviImage,extentArray,vmin,cmap):
    ndvi = gdal.Open(ndviImage)
    ds2019 = ndvi.ReadAsArray()
    plt.figure(figsize=(20,15))
    im = plt.imshow(ds2019, vmin=vmin, cmap=cmap, extent=extentArray)#
    plt.colorbar(im, fraction=0.015)
    plt.xlabel('Este')
    plt.ylabel('Norte')
    plt.show()
    

In [ ]:
imgs = []
path_B3=[]
path_B4=[]
path = "../137044-L7/"
df={}
i=0
fixedX1 = 119985
fixedX2 = 346215  # Lowest final X
fixedY1 = 2663115
fixedY2 = 2453685

for f in os.listdir(path):
     if fnmatch.fnmatch(f, '*B4*'):
        path_B4 = path + str(f)
        pattern = 'LE07_L2SP_137044_{Dates}_{otherstuff}.tif'
        result = parse(pattern, f)
        path_B3 = path + f.replace("B4.TIF", "B3.TIF")  
               
        #Output NDVI Rasters 
        path_NDVI = '../Output/NDVI.tif'
      
        #NDVI Contours
        contours_NDVIChange_19_14 = '../Output/NDVIChange_19_14.shp'
        
        #Open raster bands
        B4 = gdal.Open(path_B4)
        B3 = gdal.Open(path_B3)
        #Read bands as matrix arrays
        B4_Data = B4.GetRasterBand(1).ReadAsArray().astype(np.float32)
        B3_Data = B3.GetRasterBand(1).ReadAsArray().astype(np.float32)
        
        
        geotransform=B4.GetGeoTransform()
        originX,pixelWidth,empty,finalY,empty2,pixelHeight=geotransform
        cols =  B4.RasterXSize
        rows =  B4.RasterYSize
        
        projection = B4.GetProjection()
        finalX = originX + pixelWidth * cols
        originY = finalY + pixelHeight * rows
        
        initialrow = int((finalY-fixedY1)/30)
        finalrow = int(((finalY-originY)/30)-((fixedY2-originY)/30))
        initialcol = int((fixedX1 - originX)/30)
        finalcol = int(((finalX-originX)/30)-((finalX-fixedX2)/30))
        B4_Data = B4_Data[initialrow: finalrow,initialcol : finalcol]
        B3_Data = B3_Data[initialrow: finalrow,initialcol : finalcol]  
        
        B4_Data = B4_Data[1000: 6000,1500 : 6000]
        B3_Data = B3_Data[1000: 6000,1500 : 6000]
          
        new_rr_shape=(64,64)
        B3_Data = st.resize(B3_Data, new_rr_shape, mode='constant') 
        B4_Data = st.resize(B4_Data, new_rr_shape, mode='constant')
        ndvi = np.divide(B4_Data - B3_Data, B4_Data+ B3_Data,where=(B4_Data - B3_Data)!=0)
        ndvi[ndvi == 0] = -999
        df[result['Dates']] = ndvi
        print(result['Dates'])
        

19991124
20000228
20001126
20020201
20020217
20020305
20021116
20021202
20030119
20030308
20030324
20031119
20031205
20041020
20041207
20050108
20050225
20051210
20051226


In [ ]:
outfile1='dict64L7.npz'
np.savez(outfile, **df)

In [ ]:

#Input Raster and Vector Paths

#All images of 17044 scene

imgs = []
path_B4=[]
path_B5=[]
path = "../137044/"
df={}
i=0

# Defining Image Size
fixedX1 = 119985
fixedX2 = 346215  # Lowest final X
fixedY1 = 2663115
fixedY2 = 2453685

z=0
for f in os.listdir(path):
     if fnmatch.fnmatch(f, '*B5*'):
        path_B5 = path + str(f)
        pattern = 'LC08_L2SP_137044_{Dates}_{otherstuff}.tif'
        result = parse(pattern, f)
        path_B4 = path + f.replace("B5.TIF", "B4.TIF")  
        

        #Output NDVI Rasters 
        path_NDVI = '../Output/NDVI.tif'
      
        #NDVI Contours
        contours_NDVIChange_19_14 = '../Output/NDVIChange_19_14.shp'
        
        #Open raster bands
        B5 = gdal.Open(path_B5)
        B4 = gdal.Open(path_B4)
        #Read bands as matrix arrays
        B5_Data = B5.GetRasterBand(1).ReadAsArray().astype(np.float32)
        B4_Data = B4.GetRasterBand(1).ReadAsArray().astype(np.float32)
        
    
       
        geotransform=B5.GetGeoTransform()
        originX,pixelWidth,empty,finalY,empty2,pixelHeight=geotransform
        cols =  B5.RasterXSize
        rows =  B5.RasterYSize
        
        projection = B5.GetProjection()
        finalX = originX + pixelWidth * cols
        originY = finalY + pixelHeight * rows
        
        print("final x and originY ",finalX, originY)
        print(B5.GetGeoTransform())
        
        
        initialrow = int((finalY-fixedY1)/30)
        finalrow = int(((finalY-originY)/30)-((fixedY2-originY)/30))
        initialcol = int((fixedX1 - originX)/30)
        finalcol = int(((finalX-originX)/30)-((finalX-fixedX2)/30))
        B5_Data = B5_Data[initialrow: finalrow,initialcol : finalcol]
        B4_Data = B4_Data[initialrow: finalrow,initialcol : finalcol]  
        
        B5_Data = B5_Data[1000: 6000,1500 : 6000]
        B4_Data = B4_Data[1000: 6000,1500 : 6000]
          
        new_rr_shape=(64,64)
        B4_Data = st.resize(B4_Data, new_rr_shape, mode='constant') 
        B5_Data = st.resize(B5_Data, new_rr_shape, mode='constant')
        ndvi = np.divide(B5_Data - B4_Data, B5_Data+ B4_Data,where=(B5_Data - B4_Data)!=0)
        ndvi[ndvi == 0] = -999
        df[result['Dates']] = ndvi
        print(result['Dates'])
        

In [ ]:
outfile2='dict64L8.npz'
np.savez(outfile, **df)